In [1]:
# !pip install sranodec

In [1]:
import pandas as pd
import numpy as np
import main_classificaiton as mc
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import os
import random


In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. w/o data representation & RNN model 
config1 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'RNN', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 200 # 학습 시 사용할 epoch 수
            }
}

In [4]:
# Case 2. w/o data representation &LSTM model 
config2 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'LSTM', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 200 # 학습 시 사용할 epoch 수
            }
}

In [5]:
# Case 3. w/o data representation & GRU model 
config3 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'GRU', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 200 # 학습 시 사용할 epoch 수
            }
}

In [6]:
# Case 4. w/o data representation & CNN_1D model 
config4 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'CNN_1D', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'output_channels' : 64, # convolution channel size of output
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'kernel_size' : 3, # convolutional filter size
            'stride' : 1, # stride of the convolution. Default = 1 
            'padding' : 0, # padding added to both sides of the input. Default = 0
            'batch_size' : 64, # batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 200 # 학습 시 사용할 epoch 수
            }
}

In [24]:
# Case 5. w data representation & fully-connected layers 
# 특징 벡터의 사이즈 = 30 이라고 가정

config5 = {
        'with_representation': True, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'algorithm': 'FC_Layer', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_Layer'} 중 택 1

        'alg_parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'batch_size' : 64, # batch size
            'bias': True, # bias [True, False]
            'num_epochs' : 200 # 학습 시 사용할 epoch 수
            }
}

In [25]:
data_dir = './data'

train_x = pickle.load(open(os.path.join(data_dir, 'x_train.pkl'), 'rb'))
train_y = pickle.load(open(os.path.join(data_dir, 'state_train.pkl'), 'rb'))
test_x =  pickle.load(open(os.path.join(data_dir, 'x_test.pkl'), 'rb'))
test_y = pickle.load(open(os.path.join(data_dir, 'state_test.pkl'), 'rb'))

# train_data = {'x' : train_x, 'y' : train_y}
# test_data = {'x' : test_x, 'y' : test_y}

In [9]:
# Case 1. w/o data representation & RNN
config = config1
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/200
----------
train Loss: 1.7534 Acc: 0.1250
val Loss: 1.6363 Acc: 0.2400

Epoch 2/200
----------
train Loss: 1.6556 Acc: 0.2125
val Loss: 1.5519 Acc: 0.4400

Epoch 3/200
----------
train Loss: 1.5757 Acc: 0.4125
val Loss: 1.4809 Acc: 0.5200

Epoch 4/200
----------
train Loss: 1.5100 Acc: 0.4250
val Loss: 1.4213 Acc: 0.5200

Epoch 5/200
----------
train Loss: 1.4546 Acc: 0.4375
val Loss: 1.3730 Acc: 0.5200

Epoch 6/200
----------
train Loss: 1.4105 Acc: 0.4125
val Loss: 1.3356 Acc: 0.5200

Epoch 7/200
----------
train Loss: 1.3754 Acc: 0.4250
val Loss: 1.3068 Acc: 0.5200

Epoch 8/200
----------
train Loss: 1.3448 Acc: 0.4250
val Loss: 1.2837 Acc: 0.5200

Epoch 9/200
----------
train Loss: 1.3204 Acc: 0.4250
val Loss: 1.2664 Acc: 0.5200

Epoch 10/200
----------
train Loss: 1.2967 Acc: 0.4500
val Loss: 1.2545 Acc: 0.5200

Epoch 11/200
----------
train Loss: 1.2747 Acc: 0.4625
val Loss: 1.2456 Acc: 0.5200

Epoch 12/200
----------
train Loss: 1.2561 Acc: 0.5000
val Loss: 1.2390 Ac

In [10]:
# Case 2. w/o data representation & LSTM
config = config2
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/200
----------
train Loss: 1.7932 Acc: 0.1250
val Loss: 1.7598 Acc: 0.1200

Epoch 2/200
----------
train Loss: 1.7680 Acc: 0.1250
val Loss: 1.7349 Acc: 0.1600

Epoch 3/200
----------
train Loss: 1.7443 Acc: 0.1875
val Loss: 1.7111 Acc: 0.5600

Epoch 4/200
----------
train Loss: 1.7210 Acc: 0.3375
val Loss: 1.6875 Acc: 0.5600

Epoch 5/200
----------
train Loss: 1.6985 Acc: 0.4000
val Loss: 1.6636 Acc: 0.5600

Epoch 6/200
----------
train Loss: 1.6775 Acc: 0.4250
val Loss: 1.6401 Acc: 0.5600

Epoch 7/200
----------
train Loss: 1.6579 Acc: 0.4500
val Loss: 1.6178 Acc: 0.5200

Epoch 8/200
----------
train Loss: 1.6381 Acc: 0.4375
val Loss: 1.5971 Acc: 0.5200

Epoch 9/200
----------
train Loss: 1.6198 Acc: 0.4250
val Loss: 1.5773 Acc: 0.5200

Epoch 10/200
----------
train Loss: 1.6016 Acc: 0.4250
val Loss: 1.5576 Acc: 0.5200

Epoch 11/200
----------
train Loss: 1.5838 Acc: 0.4250
val Loss: 1.5375 Acc: 0.5200

Epoch 12/200
----------
train Loss: 1.5661 Acc: 0.4375
val Loss: 1.5176 Ac

In [11]:
# Case 3. w/o data representation & GRU
config = config3
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/200
----------
train Loss: 1.7855 Acc: 0.0500
val Loss: 1.7243 Acc: 0.4800

Epoch 2/200
----------
train Loss: 1.7273 Acc: 0.3750
val Loss: 1.6638 Acc: 0.5200

Epoch 3/200
----------
train Loss: 1.6731 Acc: 0.4250
val Loss: 1.6118 Acc: 0.5200

Epoch 4/200
----------
train Loss: 1.6282 Acc: 0.4375
val Loss: 1.5650 Acc: 0.5200

Epoch 5/200
----------
train Loss: 1.5874 Acc: 0.4250
val Loss: 1.5249 Acc: 0.5200

Epoch 6/200
----------
train Loss: 1.5493 Acc: 0.4250
val Loss: 1.4893 Acc: 0.5200

Epoch 7/200
----------
train Loss: 1.5165 Acc: 0.4250
val Loss: 1.4561 Acc: 0.5200

Epoch 8/200
----------
train Loss: 1.4885 Acc: 0.4125
val Loss: 1.4267 Acc: 0.5200

Epoch 9/200
----------
train Loss: 1.4621 Acc: 0.4125
val Loss: 1.4012 Acc: 0.5200

Epoch 10/200
----------
train Loss: 1.4392 Acc: 0.4250
val Loss: 1.3788 Acc: 0.5200

Epoch 11/200
----------
train Loss: 1.4178 Acc: 0.4250
val Loss: 1.3595 Acc: 0.5200

Epoch 12/200
----------
train Loss: 1.3986 Acc: 0.4375
val Loss: 1.3423 Ac

In [12]:
# Case 4. w/o data representation & CNN_1D
config = config4
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/200
----------
train Loss: 1.7253 Acc: 0.2875
val Loss: 1.6561 Acc: 0.6000

Epoch 2/200
----------
train Loss: 1.6451 Acc: 0.4875
val Loss: 1.5890 Acc: 0.5600

Epoch 3/200
----------
train Loss: 1.5900 Acc: 0.4250
val Loss: 1.5247 Acc: 0.5200

Epoch 4/200
----------
train Loss: 1.5212 Acc: 0.4125
val Loss: 1.4663 Acc: 0.5200

Epoch 5/200
----------
train Loss: 1.4587 Acc: 0.4125
val Loss: 1.4113 Acc: 0.5200

Epoch 6/200
----------
train Loss: 1.3882 Acc: 0.4375
val Loss: 1.3606 Acc: 0.5200

Epoch 7/200
----------
train Loss: 1.3365 Acc: 0.4125
val Loss: 1.3181 Acc: 0.5200

Epoch 8/200
----------
train Loss: 1.2896 Acc: 0.4125
val Loss: 1.2857 Acc: 0.5200

Epoch 9/200
----------
train Loss: 1.2874 Acc: 0.4125
val Loss: 1.2642 Acc: 0.5200

Epoch 10/200
----------
train Loss: 1.2248 Acc: 0.4250
val Loss: 1.2512 Acc: 0.5200

Epoch 11/200
----------
train Loss: 1.1890 Acc: 0.4125
val Loss: 1.2398 Acc: 0.5200

Epoch 12/200
----------
train Loss: 1.2047 Acc: 0.4250
val Loss: 1.2286 Ac

In [26]:
# Case 5. w/ data representation & fully-connected layers
config = config5
data_classification = mc.Classification(config, train_x, train_y, test_x, test_y)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

UnboundLocalError: local variable 'model' referenced before assignment